# do a transitive query of a wikidata concept and search CU Experts elastic endpoint for all people who have this concept in their elastic document

### Use anaconda shell to add dependencies. eg: conda install -c conda-forge sparqlwrapper

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import namedtuple
import json
import requests

endpoint_url = "https://query.wikidata.org/sparql"
ES_person_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search?size=5000"
ES_pub_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search"
headers = {'Content-type': 'application/json'}
search_term = "artificial intelligence"
person_termlist = dict()

# Find the wikidata entity where the wikidata entity label matches the search_term string AND it's a concept - Q151885
# Then find all subclasses of that concept term in wikidata
query = """SELECT ?subitem ?subitemLabel
WHERE {

  ?item ?label \"""" + search_term + """\"@en.
    ?item wdt:P279* wd:Q151885
  ?subitem wdt:P279+ ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". }
  } """



In [6]:
def WdataSearch(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [7]:
def ESsearch(uri, term):
    """Simple Elasticsearch Query"""
    query = json.dumps({
        "query": {
#            "match": {
#                "researchArea.name.exact": term
#            }
             "query_string": { "query": term }
        }
    })
    response = requests.get(uri, headers=headers, data=query)
    results = json.loads(response.text)
    return results



In [8]:
print ("Only displaying results with a search score > .5")
print (search_term)
ESresults = ESsearch(ES_person_endpoint_url,search_term)
for eresult in ESresults['hits']['hits']:
      if eresult["_score"] > .5:
        #person=personterms(search_term,eresult["_score"])
        #person.name = eresult["_source"]["name"]
        termlist=[]
        termscore={search_term,eresult["_score"]}
        termlist.append(termscore)
        person_termlist[eresult["_source"]["name"]]=termlist
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])

Wresults = WdataSearch(endpoint_url, query)


for result in Wresults["results"]["bindings"]:
   print ("\n")
   print (result['subitemLabel']['value'], result['subitem']['value'])
   ESresults = ESsearch(ES_person_endpoint_url,result['subitemLabel']['value'])
   print ("Total people who match in CU Experts: ", ESresults['hits']['total'])
   for eresult in ESresults['hits']['hits']:
      if eresult["_score"] > .5:
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])
        termscore={result['subitemLabel']['value'],eresult["_score"]}
        if eresult["_source"]["name"] in person_termlist:
            person_termlist[eresult["_source"]["name"]].append(termscore)
        else:
            termlist=[]
            termlist.append(termscore)
            person_termlist[eresult["_source"]["name"]]=termlist

print ("\nList of persons and thier terms")
for result in person_termlist:
    print(result,": ")
    for terms in person_termlist[result]:
       print("\t", terms)


Only displaying results with a search score > .5
artificial intelligence
12.566562 Bradley, Liz https://experts.colorado.edu/individual/fisid_100546
12.393734 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
12.011862 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
10.518569 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
9.880764 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
9.669895 Hulden, Mans https://experts.colorado.edu/individual/fisid_154602
9.341316 Yeh, Tom https://experts.colorado.edu/individual/fisid_151584
9.087057 Martin, James H https://experts.colorado.edu/individual/fisid_100495
9.020664 Repenning, Alexander https://experts.colorado.edu/individual/fisid_104946
8.769448 Tan, Chenhao https://experts.colorado.edu/individual/fisid_158938


machine learning http://www.wikidata.org/entity/Q2539
Total people who match in CU Experts:  66
7.2657743 Tsai, Janet Yi-Jen https://experts.colorado.ed

Total people who match in CU Experts:  0


artificial general intelligence http://www.wikidata.org/entity/Q2264109
Total people who match in CU Experts:  89
12.566562 Bradley, Liz https://experts.colorado.edu/individual/fisid_100546
12.393734 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
12.011862 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
10.518569 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
9.880764 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
9.669895 Hulden, Mans https://experts.colorado.edu/individual/fisid_154602
9.341316 Yeh, Tom https://experts.colorado.edu/individual/fisid_151584
9.087057 Martin, James H https://experts.colorado.edu/individual/fisid_100495
9.020664 Repenning, Alexander https://experts.colorado.edu/individual/fisid_104946
8.769448 Tan, Chenhao https://experts.colorado.edu/individual/fisid_158938


knowledge representation and reasoning http://www.wikidata.org/entit

Total people who match in CU Experts:  59
5.9976735 Chasteen, Stephanie Viola https://experts.colorado.edu/individual/fisid_145183
5.674554 Jennings, Tracy M https://experts.colorado.edu/individual/fisid_128765
4.856074 Zigmond, Rosalyn H https://experts.colorado.edu/individual/fisid_127550
4.5542164 Root, David H https://experts.colorado.edu/individual/fisid_159444
4.4218817 Kim, Sangbok https://experts.colorado.edu/individual/fisid_149220
4.374052 Tsai, Janet Yi-Jen https://experts.colorado.edu/individual/fisid_156447
4.19893 Jurow, A. Susan https://experts.colorado.edu/individual/fisid_129478
4.0990796 Jones, Matt https://experts.colorado.edu/individual/fisid_144611
4.0925074 Curran, Tim https://experts.colorado.edu/individual/fisid_118454
4.007343 Duren, Ron G, Jr. https://experts.colorado.edu/individual/fisid_157263


chemometrics http://www.wikidata.org/entity/Q910067
Total people who match in CU Experts:  0


semi-supervised learning http://www.wikidata.org/entity/Q1041418
Total

Total people who match in CU Experts:  59
5.9976735 Chasteen, Stephanie Viola https://experts.colorado.edu/individual/fisid_145183
5.674554 Jennings, Tracy M https://experts.colorado.edu/individual/fisid_128765
4.856074 Zigmond, Rosalyn H https://experts.colorado.edu/individual/fisid_127550
4.5542164 Root, David H https://experts.colorado.edu/individual/fisid_159444
4.4218817 Kim, Sangbok https://experts.colorado.edu/individual/fisid_149220
4.374052 Tsai, Janet Yi-Jen https://experts.colorado.edu/individual/fisid_156447
4.19893 Jurow, A. Susan https://experts.colorado.edu/individual/fisid_129478
4.0990796 Jones, Matt https://experts.colorado.edu/individual/fisid_144611
4.0925074 Curran, Tim https://experts.colorado.edu/individual/fisid_118454
4.007343 Duren, Ron G, Jr. https://experts.colorado.edu/individual/fisid_157263


decision tree learning http://www.wikidata.org/entity/Q16766476
Total people who match in CU Experts:  68
8.409035 Svoboda, John D. https://experts.colorado.edu/indi

Total people who match in CU Experts:  66
5.9976735 Chasteen, Stephanie Viola https://experts.colorado.edu/individual/fisid_145183
5.674554 Jennings, Tracy M https://experts.colorado.edu/individual/fisid_128765
4.856074 Zigmond, Rosalyn H https://experts.colorado.edu/individual/fisid_127550
4.5542164 Root, David H https://experts.colorado.edu/individual/fisid_159444
4.4218817 Kim, Sangbok https://experts.colorado.edu/individual/fisid_149220
4.374052 Tsai, Janet Yi-Jen https://experts.colorado.edu/individual/fisid_156447
4.285972 Holzinger, Marcus J https://experts.colorado.edu/individual/fisid_164054
4.19893 Jurow, A. Susan https://experts.colorado.edu/individual/fisid_129478
4.0990796 Jones, Matt https://experts.colorado.edu/individual/fisid_144611
4.0925074 Curran, Tim https://experts.colorado.edu/individual/fisid_118454


automated machine learning http://www.wikidata.org/entity/Q43967068
Total people who match in CU Experts:  66
7.2657743 Tsai, Janet Yi-Jen https://experts.colorado

...

Total people who match in CU Experts:  16
7.3805046 Hewitt, Christina https://experts.colorado.edu/individual/fisid_101528
5.0817986 Link, Chris https://experts.colorado.edu/individual/fisid_109073
4.9293838 Arnoult, Nausica Christine https://experts.colorado.edu/individual/fisid_164094
4.7558084 Olwin, Brad https://experts.colorado.edu/individual/fisid_109888
4.637385 Wadsworth, Sally Jo https://experts.colorado.edu/individual/fisid_102855
4.1878357 Jones, Kevin R. https://experts.colorado.edu/individual/fisid_102094
4.1878357 Chen, Zhe https://experts.colorado.edu/individual/fisid_148754
4.1826305 Eckert, Carrie Ann https://experts.colorado.edu/individual/fisid_154026
4.087283 Willcutt, Erik G https://experts.colorado.edu/individual/fisid_113861
3.837353 Boswell, Robert E https://experts.colorado.edu/individual/fisid_100196


kernel perceptron http://www.wikidata.org/entity/Q17093018
Total people who match in CU Experts:  0


local beam search http://www.wikidata.org/entity/Q58945661

	 {6.352607, 'recursive neural network'}
	 {'generative adversarial network', 6.352607}
	 {'memory-augmented neural network', 6.352607}
	 {'deep neural network', 6.352607}
	 {'graph convolutional network', 6.352607}
	 {'Jordan Network', 6.352607}
	 {'Elman Network', 6.352607}
	 {'Probabilistic neural network', 6.352607}
	 {'convolutional neural network', 6.352607}
	 {'highway network', 6.352607}
Eargle, David : 
	 {5.655576, 'artificial neural network'}
	 {5.655576, 'Hopfield network'}
	 {'recurrent neural network', 5.655576}
	 {5.655576, 'Radial basis function network'}
	 {5.655576, 'modular neural network'}
	 {5.655576, 'Kohonen neural network'}
	 {'compositional pattern-producing network', 5.655576}
	 {'feedforward neural network', 5.655576}
	 {5.655576, 'recursive neural network'}
	 {'generative adversarial network', 5.655576}
	 {'memory-augmented neural network', 5.655576}
	 {'deep neural network', 5.655576}
	 {'graph convolutional network', 5.655576}
	 {'Jordan Network', 5.655576

	 {6.147673, 'computer-based question classification'}
	 {'document classification', 6.147673}
Kalwara, Jim : 
	 {6.124983, 'statistical classification'}
	 {6.124983, 'computer-based question classification'}
	 {'document classification', 6.124983}
Eberle, Jaelyn J : 
	 {5.9838614, 'statistical classification'}
	 {5.9838614, 'computer-based question classification'}
	 {'document classification', 5.9838614}
Moeller, Paul D. : 
	 {5.4901, 'statistical classification'}
	 {5.4901, 'computer-based question classification'}
	 {'document classification', 5.4901}
Selden, Karen E : 
	 {5.276823, 'statistical classification'}
	 {5.276823, 'computer-based question classification'}
	 {'document classification', 5.276823}
Ferris, Anna M : 
	 {5.2237806, 'statistical classification'}
	 {5.2237806, 'computer-based question classification'}
	 {'document classification', 5.2237806}
Desert, Jean-Michel : 
	 {4.53579, 'statistical classification'}
	 {4.53579, 'computer-based question classification'}
	 {

	 {'cognitive anthropology', 6.654515}
Bonino, Angela Yarnell : 
	 {8.252977, 'Cognitive hearing science'}
Baiduc, Rachael Rebecca : 
	 {'Cognitive hearing science', 7.5339427}
Schick, Brenda : 
	 {'Cognitive hearing science', 7.5134716}
Dwyer, Michael Benjamin : 
	 {'cognitive geography', 7.5305676}
Ortman, Scott Graham : 
	 {'cognitive geography', 7.432549}
Oakes, Timothy S : 
	 {'cognitive geography', 6.742798}
Barnard, Holly Rene : 
	 {'cognitive geography', 6.654515}
Lininger, Katherine : 
	 {'cognitive geography', 6.654515}
Blanken, Peter David : 
	 {'cognitive geography', 6.2663007}
	 {'sentence boundary disambiguation', 5.059114}
Bryan, Joe : 
	 {'cognitive geography', 6.2663007}
Perigo, Levi : 
	 {4.973447, 'Hopfield network'}
	 {'recurrent neural network', 4.973447}
	 {4.973447, 'Radial basis function network'}
	 {4.973447, 'modular neural network'}
	 {4.973447, 'Kohonen neural network'}
	 {'compositional pattern-producing network', 4.973447}
	 {'feedforward neural network', 

Boyd, Samuel L : 
	 {8.015345, 'winner-take-all'}
DeFelice, Thomas Peter : 
	 {'winner-take-all', 4.6574826}
Long, Rong : 
	 {'Long Short-Term Memory', 6.6133723}
Long, Daniel Robert : 
	 {5.9241405, 'Long Short-Term Memory'}
Long, Yicheng : 
	 {5.664014, 'Long Short-Term Memory'}
Long, Mary K : 
	 {5.1360607, 'Long Short-Term Memory'}
Jordan, Judith Elena : 
	 {'Jordan Network', 5.9956546}
Jordan, Allen Franklin : 
	 {'Jordan Network', 5.9487844}
Adams, Jordan Marie : 
	 {'Jordan Network', 5.9094486}
Tewksbury, Joshua Jordan : 
	 {'Jordan Network', 5.165747}
Diduch, Paul Jordan : 
	 {'Jordan Network', 5.165747}
Piper, Mark : 
	 {'sentence boundary disambiguation', 6.034531}
Stammerjohn, Sharon E : 
	 {'sentence boundary disambiguation', 5.607352}
Persson, Ola P G : 
	 {'sentence boundary disambiguation', 5.505552}
Shupe, Matthew D : 
	 {'sentence boundary disambiguation', 4.6672325}
Wang, Zhien : 
	 {'sentence boundary disambiguation', 4.4263234}
Calkins, Michael Andrew : 
	 {'sentenc